In [1]:
import numpy as np
import patsy
import statsmodels.api as sm
from scipy import stats

# Exercise 1

In [2]:
def f(x):
    return stats.norm.pdf(x[:,0]-0.5, scale=0.2) * stats.norm.pdf(x[:,1]-0.5, scale=0.2)

In [3]:
n = 10000
X = np.random.uniform(size=(1000, 2))
y = f(X) + np.random.randn(1000)*0.4
knots1 = [1/4, 2/4, 3/4]
deg = 3

In [4]:
def bx_bspline(x, m):
    knots = [(i/(m+1)) for i in range(1, m+1)]
    return patsy.bs(x=x, knots=knots, degree=3, include_intercept=True, lower_bound=0, upper_bound=1)

def bx_tensor(x, m):
    basis_num = m + 1 + 3
    bx1 = bx_bspline(x=x[:, 0], m=m)
    bx2 = bx_bspline(x=x[:, 1], m=m)
    tmp_list = [bx1[:,i].reshape((-1,1))*bx2 for i in range(basis_num)]
    output = np.hstack(tmp_list)
    return output

def get_fhat(x, y, m):
    bx = bx_tensor(x=x, m=m)
    coef = sm.OLS(y, bx).fit().params
    fhat = bx @ coef
    return fhat

In [5]:
fhat = get_fhat(X, y, 3)

In [6]:
np.mean((fhat-f(X))**2)

0.007093651213111497

# Exercise 2

In [7]:
def bx_tensor2(x, m):
    basis_num = m + 1 + 3
    bx1 = bx_bspline(x=x[:, 0], m=m)
    bx2 = bx_bspline(x=x[:, 1], m=m)
    output = np.hstack((bx1, bx2))
    return output[:, :-1]

def get_fhat2(x, y, m):
    bx = bx_tensor2(x=x, m=m)
    coef = sm.OLS(y, bx).fit().params
    fhat = bx @ coef
    return fhat

In [8]:
fhat2 = get_fhat2(X, y, 3)

In [9]:
np.mean((fhat2-f(X))**2)

0.18694411961673812